In [1]:
!pip install praw

    100% |████████████████████████████████| 143kB 1.0MB/s ta 0:00:01
    100% |████████████████████████████████| 204kB 1.3MB/s ta 0:00:01
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import praw #import praw - a module to scrape information from reddit

In [3]:
reddit = praw.Reddit(client_id='FIvso8ETavDmfw', client_secret='uZy6eZ6fs7PJEblJO2lwbxs1f_U', user_agent='project_script')


In [14]:
flairs = ["Scheduled", "Politics", "Photography", "Policy/Economy", "AskIndia" , "Sports", "Non-Political" ,"Science/Technology" ,"FoodBusiness/Finance" ,"Coronavirus", "CAA-NRC-NPR"]

In [4]:
subreddit = reddit.subreddit('india')

## Attributes of reddit submission class that we consider in the code:
**title** -->	The title of the submission.  
**score** -->	The number of upvotes for the submission.  
**selftext** -->	The submissions’ selftext - an empty string if a link post.  
**url** -->	The URL the submission links to, or the permalink if a selfpost.  
**num_comments** -->	The number of comments on the submission.  
**comments** -->	Provides an instance of CommentForest.  
**flair** --> flair of the submission.  
**id** -->	ID of the submission.  

info regarding other attributes can be found at - https://praw.readthedocs.io/en/latest/code_overview/models/submission.html


In [44]:
# Creating a dictionary of reddit submissions under r/india 
# The dictionary has 5 keys title, score, body, url, num_comments, comments, flair, id
topics_dict = {"title":[],"score":[],"body":[],"url":[],"num_comments":[],"comments":[],"flair":[],"id":[]}
for flair in flairs:                                           # Loop over all flairs
    submissions = subreddit.search(flair,limit = 1000)         # Extracting list of submissions for each flair
    for submission in submissions:                             # Loop to append each submission's attributes to its corresponding list
        topics_dict["flair"].append(flair)
        topics_dict["title"].append(submission.title)
        topics_dict["score"].append(submission.score)
        topics_dict["id"].append(submission.id)
        topics_dict["url"].append(submission.url)
        topics_dict["num_comments"].append(submission.num_comments)
        topics_dict["body"].append(submission.selftext)
        
        submission.comments.replace_more(limit=None)
        comment = ''
        for top_level_comment in submission.comments:          # Extracting top level comments for each submission
            comment = comment + ' ' + top_level_comment.body
        
        topics_dict["comments"].append(comment)
       
   
topics_data = pd.DataFrame(topics_dict)


NameError: name 'pd' is not defined

In [52]:
import pickle as pkl                      # Saving topics_dict as pickle file for quick access in future
with open('topics_dict.pkl','wb') as f:
    pkl.dump(topics_dict,f)

In [63]:
import pandas as pd
topics_data = pd.DataFrame.from_dict(topics_dict)

In [48]:
topics_data.to_csv('reddit-data.csv', index=False)

In [68]:
from IPython.display import display, HTML

display(HTML(topics_data.to_html()))

,title,score,body,url,num_comments,comments,flair,id
0,Delhi Govt Sources: Names of CM Arvind Kejriwa...,302,,https://twitter.com/ANI/status/123109390051893...,30,This is beyond petty. > The inclusion of a De...,Scheduled,f7ogd8
1,"Delhi: AP Singh, advocate of 2012 Delhi gang-r...",17,,https://twitter.com/ani/status/124073128907587...,22,"My hunch is , this guy is trying to expose th...",Scheduled,flgvah
2,Why the Supreme Court’s verdict on SC/ST quota...,106,,https://scroll.in/article/952687/why-the-supre...,47,Muslims and reservation are two distractions ...,Scheduled,f1o839
3,What about the entrance exams scheduled in May?,9,"CLAT, AILET, NEET, JEE have been postponed bu...",https://www.reddit.com/r/india/comments/fvcvo1...,3,"Bachega India, tabhi toh Padhega India. Gand ...",Scheduled,fvcvo1
4,Advisory: No scheduled international commercia...,36,,https://twitter.com/PIB_India/status/124061071...,4,Oh boy. Chalo bhaisahab. Sabji ka dukaan main...,Scheduled,fl8zf5
5,My roommate is in India (he's an Indian and no...,29,He was working remotely from February 24th and...,https://www.reddit.com/r/india/comments/fhgyhu...,4,"AFAIK, U.S. still hasn't banned entries of In...",Scheduled,fhgyhu
6,No ministry scores 100% on fund utilisation fo...,48,,https://m.economictimes.com/news/politics-and-...,1,But we are funding Sanskrit Universities with...,Scheduled,fg8v3u
7,"Hydroxychloroquine now a schedule H1 drug, can...",29,,https://www.mohfw.gov.in/pdf/218927g.pdf,4,Someone give me a ELI5? What's this drug used...,Scheduled,fpr0on
8,Maharashtra govt: All schools in urban areas o...,30,,https://mobile.twitter.com/ANI/status/12388139...,4,Bc students in 12 and 10 are not immune to th...,Scheduled,fii4k1
9,[AskIndia] Indigo cancelled flight less than 8...,14,I had Indigo flight from HYD-BOM on 14th Dec @...,https://www.reddit.com/r/india/comments/egtg1j...,12,Yes I think you are legally eligible to claim...,Scheduled,egtg1j


## Out of many, I chose only these attributes as they might contain information to distinguish between different flairs 
## for example --

In [154]:
print(topics_data.iloc[243]['flair'],"--->",topics_data.iloc[243]['title'],end = '\n\n')
print(topics_data.iloc[483]['flair'],"--->",topics_data.iloc[483]['title'])

Politics ---> No soft Hindutva, no Left Revolution, Kejriwal establishing a new centre in Indian politics

Photography ---> [OC] Just messing around with my camera two days ago. Complete newbie to photography and image processing. What do you think of this image of a rather photogenic cloud and the moon? [Kolkata] [Sony a6000+E35mm f/1.8 lens] [ISO100, f/10.0, 1/8s exposure, 6020x4023]


### Above, I have taken the example of two random submissions and printed there titles
  
One submission comes under **politics** and other in **Photography** flair  
- We can observe that title of **politics** has words like -- Hindutva, Revolution, Kejriwal, Indian, Politics  
- Title of **Photography** has words like -- camera, photography, image, processing, photogenic, Sony, f/1.8, exposure, lens
  
Similarly, other attributes might also contain such distinguishing features such as :-
  

## URL

In [173]:
print(topics_data.iloc[243]['flair'],"--->",topics_data.iloc[243]['url'],end = '\n\n')
print(topics_data.iloc[483]['flair'],"--->",topics_data.iloc[483]['url'])

Politics ---> https://theprint.in/opinion/no-soft-hindutva-no-left-revolution-kejriwal-establishing-a-new-centre-in-indian-politics/358776/

Photography ---> https://imgur.com/SO8wJSn


## Body

In [172]:
print(topics_data.iloc[250]['flair'],"--->",topics_data.iloc[250]['body'],end = '\n\n')
print(topics_data.iloc[484]['flair'],"--->",topics_data.iloc[484]['body'])

Politics ---> Say you are disappointed with the state of affairs. You think the economy, the politics, the state, can be better managed and you have thoughts, theories and ideas on how.

So wise men and women of India, what's preventing you from taking matters into your own hands? 

Why do you allow yourself to be governed poorly (say)? Or, are you satisfied?

Photography ---> I am looking for a good film/analog camera for photography but I cannot find it anywhere online. I searched in various online camera stores and even eBay but I couldn't find anything. Can someone help with it, or any offline store/market where I can get it? I live in Pune and I don't mind traveling to Mumbai, also I don't mind getting 2nd hand stuff as long as it is in working condition.
